<a href="https://colab.research.google.com/github/EziamaUgonna/Bayesian_analysis-/blob/master/work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, keras-preprocessing, protobuf, numpy, tensorflow-estimator, termcolor, wrapt, absl-py, wheel, keras-applications, astor, tensorboard, gast, grpcio, google-pasta
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
!pip show tensorflow_probability

Name: tensorflow-probability
Version: 0.7.0
Summary: Probabilistic modeling and statistical inference in TensorFlow
Home-page: http://github.com/tensorflow/probability
Author: Google LLC
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: decorator, cloudpickle, six, numpy
Required-by: tensor2tensor, magenta, dm-sonnet


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals 

In [0]:
# Dependencies
import os 
import warnings 
#from absl import flags 
import matplotlib 
import numpy as np 
import tensorflow as tf 
import tensorflow_probability as tfp
import math
import pandas as pd
tfd = tfp.distributions
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve 

In [0]:
flags = tf.app.flags
FLAGS = tf.app.flags.FLAGS
flags.DEFINE_float("learning_rate", default = 0.0001, help = "Initial learning rate.")
flags.DEFINE_integer("epochs", default = 100, help = "Number of epochs to train for")
flags.DEFINE_integer("batch_size", default =8, help = "Batch size.")
flags.DEFINE_integer("eval_freq", default = 400, help =" Frequency at which to validate the model.")
flags.DEFINE_float("kernel_posterior_scale_mean", default = -0.9, help = "Initial kernel posterior mean of the scale (log var) for q(w)")
flags.DEFINE_float("kernel_posterior_scale_constraint", default = 0.2, help = "Posterior kernel constraint for the scale (log var) for q(w)")
flags.DEFINE_float("kl_annealing", default = 50, help = "Epochs to anneal the KL term (anneals from 0 to 1)")
flags.DEFINE_integer("num_hidden_layers", default = 4, help = "Number of hidden layers")
flags.DEFINE_integer("num_monte_carlo",

                     default=50,

                     help="Network draws to compute predictive probabilities.")
tf.app.flags.DEFINE_string('f', '', 'kernel')
#initialize flags 
#FLAGS = flags.FLAGS
print(FLAGS.learning_rate)
print(FLAGS.epochs)
print(FLAGS.num_monte_carlo)

0.0001
100
50


In [0]:
def build_input_pipeline(X_train,X_val,X_test,y_train,y_val,y_test, batch_size, valid_size):
  #Build an iterator over training batches 
  training_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  #Shuffle the dataset (note shuffle argument much larger than training size)
  # and form batches of size batch_size
  training_batches = training_dataset.shuffle(20000, reshuffle_each_iteration =True).repeat().batch(batch_size)
  training_iterator = tf.data.make_one_shot_iterator(training_batches)
  
  #Building iterator over the heldout set with batch_size = heldout_size,
  # i.e., return the entire heldout set as a constant.
  heldout_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
  heldout_batches = heldout_dataset.repeat().batch(valid_size)
  heldout_iterator = tf.data.make_one_shot_iterator(heldout_batches)
  
  test_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))
  test_dataset = test_dataset.batch(batch_size)
  test_iterator = test_dataset.make_one_shot_iterator()
  
  
  
  
  #Combine these into a feasible iterator that can switch between training 
  # and validation inputs.
  # Here should be minibatch increment be defined 
  handle = tf.placeholder(tf.string, shape = [])
  feedable_iterator = tf.data.Iterator.from_string_handle(handle, training_batches.output_types, training_batches.output_shapes)
  features_final, labels_final = feedable_iterator.get_next()
  
  return features_final, labels_final, handle, training_iterator, heldout_iterator,test_iterator


In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls "/content/gdrive/My Drive/"

 acc_1.csv		     elvin.csv
 acc_2.csv		    'Getting started.pdf'
 acc_3.csv		     kernel.ipynb
 acc_4.csv		     preds_2.npy
 acc_5.csv		     preds_3.npy
 Bayesian_log_trial_.ipynb   Untitled0.ipynb
 BNN_Work.ipynb		     Untitled1.ipynb
 BSM11.csv		     Variational_autoencoder-1-1.ipynb
 BSM3.csv		     work1.csv
'Colab Notebooks'	     work2.csv
 data1.csv		    'work _desertation.ipynb'
 data23.csv		     work_main


In [0]:
# Read in the dataset
df = pd.read_csv('/content/gdrive/My Drive/work2.csv').astype(np.float32)
change = df.query('Speed>0').sample(frac = .1).index
df.loc[change, 'Speed'] = 0
df.loc[change, 'Class'] = 0
df.to_csv('work2.csv', header = True, index =False)


In [0]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=200)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=200)

In [0]:
# Standardize the dataset 
scalar_x_train = StandardScaler().fit(X_train)
X_train = scalar_x_train.transform(X_train)
X_test  = scalar_x_train.transform(X_test)
X_val   = scalar_x_train.transform(X_val)


In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15229799517696383394, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15973839183552396903
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14480806543537414401
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14892338381
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8109248510153624182
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [0]:
def main(argv):
  # extract the activation function from the hyperopt spec as an attribute from the tf.nn module 
  #activation = getattr(tf.nn, FLAGS.activation_function)
  # define the graph 
  #with tf.Graph().as_default():
  (features_final, labels_final, handle, training_iterator, heldout_iterator,test_iterator) = build_input_pipeline(X_train,X_val,X_test,y_train,y_val,y_test, FLAGS.batch_size, 500)
  
  
  # Building the Bayesian Neural Network 
  # we are Gaussian Reparametrization Trick 
  # to compute the stochastic gradients as described in the paper 
  with tf.compat.v1.name_scope("bayesian_neural_net", values =[features_final]):
    neural_net = tf.keras.Sequential()
    for i in range(FLAGS.num_hidden_layers):
      layer = tfp.layers.DenseReparameterization(
          units = 10,
          activation = tf.nn.relu,
          trainable = True,
          kernel_prior_fn=tfp.layers.default_multivariate_normal_fn, # NormalDiag
          kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(),
          #kernel_posterior_fn=tfp_layers_util.default_mean_field_normal_fn(), # softplus(sigma)
          #kernel_posterior_tensor_fn=lambda x: x.sample(),
          bias_prior_fn=tfp.layers.default_multivariate_normal_fn, # NormalDiag
          #bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(), # softplus(sigma)
         # bias_posterior_tensor_fn=lambda x: x.sample()
          )
      neural_net.add(layer)
  neural_net.add(tfp.layers.DenseReparameterization(
      units=1, # one dimensional output
      activation= tf.nn.sigmoid, # since regression (outcome not bounded)
      trainable=True, # i.e subject to optimization
      kernel_prior_fn=tfp.layers.default_multivariate_normal_fn, # NormalDiag with hyperopt sigma
      #kernel_posterior_fn=tfp.layers.default_mean_field_normal_fn(), # softplus(sigma)
      #kernel_posterior_tensor_fn=lambda x: x.sample(),
      bias_prior_fn =tfp.layers.default_multivariate_normal_fn, # NormalDiag with hyperopt sigma
      #bias_posterior_fn=tfp.layers.default_mean_field_normal_fn(), # softplus(sigma)
      #bias_posterior_tensor_fn=lambda x: x.sample()
      ))
  logits = neural_net(features_final)
  labels_distribution = tfd.Bernoulli(logits=logits)
  #labels_distribution = tfd.Categorical(logits=logits)
  
  # Perform KL annealing. The optimal number of annealing steps 
  # depends on the dataset and architecture.
  t = tf.Variable(0.0)
  kl_regularizer = t / (FLAGS.kl_annealing * len(X_train) / FLAGS.batch_size)
  
  #Compute the -ELBO as the loss. The kl term is annealed from 1 to 1 over 
  # the epochs specified by the kl_annealing flag.
  log_likelihood = labels_distribution.log_prob(labels_final)
  #neg_log_likelihood = tf.reduce_mean(tf.squared_difference(logits,labels_final))
  neg_log_likelihood = -tf.reduce_mean(input_tensor = log_likelihood)
  kl = sum(neural_net.losses)/len(X_train) * tf.minimum(1.0, kl_regularizer)
  elbo_loss = neg_log_likelihood + kl
  
  # Build metrics for evaluation. Predictions are formed from single forward 
  # pass of the probablisitic layers . They are cheap but noisy predictions
  predictions = tf.argmax(input = logits, axis=1)
  #predictions = tf.sigmoid(logits)
  predictions = tf.cast(predictions, tf.float32)
  
  epsilon = tf.constant(value=0.0000001)
  
  tp = tf.reduce_sum(tf.cast(labels_final * predictions, 'float'), axis = 0)
  tn = tf.reduce_sum(tf.cast((1-labels_final)*(1-predictions), 'float'), axis=0)
  fp = tf.reduce_sum(tf.cast((1- labels_final) * predictions,'float'), axis =0)
  fn = tf.reduce_sum(tf.cast (labels_final * (1 - predictions), 'float'), axis =0)

  precision = tp / (tp + fp + epsilon)
  recall = tp / (tp + fn + epsilon)
  tpr = tp/(tp + fn + epsilon)
  fpr = fp/(tp + fn + epsilon)
  
  def f1_score(labels_finals,predictions):
        
    f1_1 = 2*precision*recall/(precision + recall+ epsilon)

    f1_1 = tf.where(tf.is_nan(f1_1), tf.zeros_like(f1_1), 1) 
    return tf.math.reduce_mean(f1_1)
  
  f1 = f1_score(labels_final, predictions)
  
  
  
  with tf.compat.v1.name_scope("train"):
    train_accuracy, train_accuracy_update_op = tf.metrics.accuracy(labels=labels_final,predictions =predictions)
    #opt = tf.train.RMSPropOptimizer(FLAGS.learning_rate)
    opt = tf.train.AdamOptimizer(FLAGS.learning_rate)
    #opt = tf.train.MomentumOptimizer(FLAGS.learning_rate, momentum=0.9)
    train_op = opt.minimize(elbo_loss)
    update_step_op = tf.assign(t, t+1)
  
  with tf.compat.v1.name_scope("valid"):
    valid_accuracy, validation_accuracy_update_op = tf.metrics.accuracy(labels= labels_final,predictions = predictions)
  with tf.compat.v1.name_scope("test"):
    test_accuracy, test_accuracy_update_op = tf.metrics.accuracy(labels = labels_final,predictions = predictions)
  
  init_op = tf.group(tf.global_variables_initializer(),
                     tf.local_variables_initializer())
  saver = tf.train.Saver()
  
  stream_vars_valid = [ v for v in tf.local_variables() if "valid" in v.name]
  
  reset_valid_op = tf.variables_initializer(stream_vars_valid)
  
  valid_accuracy_summary = []
  stop_early =0
  with tf.compat.v1.Session() as sess:
    sess.run(init_op)
    
    # Run the training loop

    train_handle = sess.run(training_iterator.string_handle())
    heldout_handle = sess.run(  heldout_iterator.string_handle())
      
    test_handle = sess.run(test_iterator.string_handle())

    training_steps = int(round(FLAGS.epochs * (len(X_train) / FLAGS.batch_size)))

    for step in range(training_steps):
      sess.run(train_accuracy_update_op, feed_dict={handle: heldout_handle})

      

      #_ = sess.run([train_op,train_accuracy_update_op, update_step_op],feed_dict={handle: train_handle})
      
      # Manually print the frequency 
      if step % 100 == 0:
        save_path = saver.save(sess, "/tmp/my_model.ckpt")
        loss_value, accuracy_value, kl_value = sess.run([elbo_loss, train_accuracy, kl], feed_dict= {handle:train_handle})
        print("Step:{:>3d} loss : {:.3f} KL: {:.3f}" .format(step , loss_value, accuracy_value, kl_value))
        
        
      if (step +1) % FLAGS.eval_freq ==0: 
        # Compute log prob of heldout set by averaging draws from the model:
        # p(heldout | train) = int_model p(heldout|model) p(model|train) ~= 1/n * sum_{i=1}^n p(heldout | model_i)
        # where model_i is a draw from the posterior 
        #p(model|train)
        probs = np.asarray([sess.run((labels_distribution.probs), 
                                     feed_dict ={handle: heldout_handle})
      
                            for _ in range(FLAGS.num_monte_carlo)])
        mean_probs = np.mean(probs, axis =0).astype(np.int32)
        print(mean_probs.dtype)
        
        _, label_vals = sess.run((features_final, labels_final), feed_dict = {handle: heldout_handle})
        label_vals = (label_vals).astype(np.int32)
      
       # heldout_lp = np.mean(np.log(mean_probs[np.arange(mean_probs.shape[0]), label_vals]))
        
        
        #print(" ...Held_out nats: {:.3f}".format(heldout_lp))
        
       # Calculate validation accuracy

        for step in range(10):

          sess.run(validation_accuracy_update_op, feed_dict={handle: heldout_handle})

          valid_value = sess.run(valid_accuracy, feed_dict={handle: heldout_handle})
        
        valid_accuracy_summary.append(valid_value)
        if valid_value < max(valid_accuracy_summary) and step > 100:
          stop_early += 1
          if stop_early == 3:
            break
        else:
          stop_early = 0
        print("Validation Accuracy: {:.3f}".format(valid_value))
        sess.run(reset_valid_op)
        sess.run(test_accuracy_update_op, feed_dict={handle: test_handle})
          
        test_value = sess.run(test_accuracy, feed_dict={handle: test_handle})
        
        loss_value, precision_value, recall_value, fpr_value, tpr_value,f1_value = sess.run([elbo_loss, precision, recall, fpr, tpr,f1],feed_dict={handle: test_handle})
        
        print("Step: {:>3d} Loss: {:.3f}  test Accuracy: {:.3f} Precision: {:.3f} Recall: {:.3f} ".format(step, loss_value, test_value, precision_value, recall_value))
        
        print("Step: {:>3d} fpr: {:.3f} tpr: {:.3f} f1_1: {:.3f}".format( step, fpr_value, tpr_value,f1_value))
        
        
        
        
if __name__ == "__main__":
  tf.compat.v1.app.run()
  

W0705 20:19:39.188224 139808450086784 deprecation.py:323] From <ipython-input-7-c75a975b9106>:17: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
W0705 20:19:39.306133 139808450086784 deprecation.py:323] From <ipython-input-7-c75a975b9106>:26: DatasetV1.output_types (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
W0705 20:19:39.307211 139808450086784 deprecation.py:323] From <ipython-input-7-c75a975b9106>:26: DatasetV1.output_shapes (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future ver

Step:  0 loss : 0.599 KL: 0.104
Step:100 loss : 0.472 KL: 0.094
Step:200 loss : 0.471 KL: 0.093
Step:300 loss : 0.467 KL: 0.091
int32
Validation Accuracy: 0.087
Step:   9 Loss: 0.473  test Accuracy: 0.000 Precision: 0.000 Recall: 0.000 
Step:   9 fpr: 0.000 tpr: 0.000 f1_1: 1.000
Step:400 loss : 0.535 KL: 0.092
Step:500 loss : 0.535 KL: 0.092
Step:600 loss : 0.473 KL: 0.092
Step:700 loss : 0.535 KL: 0.092
int32
Validation Accuracy: 0.091
Step:   9 Loss: 0.471  test Accuracy: 0.000 Precision: 0.000 Recall: 0.000 
Step:   9 fpr: 0.000 tpr: 0.000 f1_1: 1.000
Step:800 loss : 0.535 KL: 0.092
Step:900 loss : 0.536 KL: 0.092
Step:1000 loss : 0.472 KL: 0.092
Step:1100 loss : 0.472 KL: 0.092
int32
Validation Accuracy: 0.090
Step:   9 Loss: 0.536  test Accuracy: 0.083 Precision: 0.000 Recall: 0.000 
Step:   9 fpr: 0.000 tpr: 0.000 f1_1: 1.000
Step:1200 loss : 0.725 KL: 0.092
Step:1300 loss : 0.473 KL: 0.092


KeyboardInterrupt: ignored

In [0]:
# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(X_train)
# apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(y_train)
# zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(8)

In [0]:
# do the same operations for the validation set
dx_valid = tf.data.Dataset.from_tensor_slices(X_test)
dy_valid = tf.data.Dataset.from_tensor_slices(y_test)
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(500).repeat().batch(30)

In [0]:
# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
next_element = iterator.get_next()

In [0]:
# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [0]:
def nn_model(in_data):
    bn = tf.layers.batch_normalization(in_data)
    fc1 = tf.layers.dense(bn, 19)
    fc2 = tf.layers.dense(fc1, 8)
    fc3 = tf.layers.dense(fc2, 2)
    return fc3

In [0]:
# create the neural network model
logits = nn_model(next_element[0])

In [0]:
# add the optimizer and loss
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=next_element[1], logits=logits))
optimizer = tf.train.AdamOptimizer(1e-8).minimize(loss)
# get accuracy
predictions = tf.argmax(logits, 1)
equality = tf.equal(predictions, tf.argmax(next_element[1], 1))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))


#TP, TN, FP, FN
TP = tf.count_nonzero(predictions * y_test, dtype=tf.float32)
TN = tf.count_nonzero((predictions-1)*(y_test-1), dtype=tf.float32)
FP = tf.count_nonzero(predictions*(y_test-1), dtype=tf.float32)
FN = tf.count_nonzero((predictions-1)*y_test, dtype=tf.float32)
#precision, recall, f1
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * precision * recall / (precision + recall)
tpr = TP/(TP+FN)
fpr = FP/(TP+FN)

init_op = tf.global_variables_initializer()

In [0]:
# run the training
epochs = 600
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc = sess.run([loss, optimizer, accuracy])
        if i % 50 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
    # now setup the validation run
    valid_iters = 1000
    # re-initialize the iterator, but this time with validation data
    sess.run(validation_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        
        loss_value, valid_accuracy, precision_value, recall_value, fpr_value, tpr_value,f1_value = sess.run([loss, accuracy, precision, recall, fpr, tpr,f1])
        print("Step: {:>3d} loss: {:.3f} Validation Accuracy: {:.3f} Precision: {:.3f} Recall: {:.3f} fpr:{:.3f} tpr:{:.3f} f1:{:.3f} ".format(
            i, loss_value, valid_accuracy, precision_value, recall_value, fpr_value, tpr_value, f1_value))
        
        
        
        
        acc = sess.run([accuracy])
        avg_acc += acc[0]
        print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters,(avg_acc ) ))   

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score,  f1_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from inspect import signature
from sklearn.metrics import brier_score_loss

# clf = svm.SVC(kernel='rbf',degree=3, probability=False,tol=0.001)
clf = RandomForestClassifier(n_estimators=100,random_state=0)
# clf = LogisticRegression(C=0.1)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
auc = roc_auc_score(y_test,y_pred)


fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=2)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(precision_score(y_test,y_pred))
print( recall_score(y_test,y_pred))
print( f1_score(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(fpr, tpr)
print("AUC:",auc)


In [0]:
v = 0 
count = 0
for v in range(len(df)):
  if df["Class"][v] == 1.0:
    count = count + 1
    

In [0]:
count

In [0]:
count

In [0]:
X_train